### *Podemos rodar às segundas-feiras já que as partidas da liga são jogadas aos domingos*

In [1]:
import pandas as pd
from functions import clube_id_e_sessao, pega_dados, conecta_ao_db

In [2]:
sessao, club_id= clube_id_e_sessao()

Clube ID: 79085


### Informações gerais dos jogos

In [3]:
# Conecta ao banco de dados
db= conecta_ao_db('dados_sokker.db')
curs= db.cursor()

#Pega informações únicas das partidas já adicionadas para que não sejam inseridos valores duplicados na base
matches = pd.read_sql_query("SELECT * FROM matches", db)

ids= matches['matchID'].unique()
data_esperada= matches['dateExpected'].unique()
data_iniciada= matches['dateStarted'].unique()

In [4]:
# Id do clube a serem pegas as informações
id_clube= club_id

# Pegando informações das partidas retirando aquelas que já foram pegas
url= f'https://sokker.org/xml/matches-team-{club_id}.xml'
df_matches= pega_dados(url, sessao, search= 'match')

#Retirando valores nulos (partidas ainda não jogadas) e duplicados (partidas já adicionadas na base)
df_matches.dropna(inplace= True)

query= 'matchID not in @ids and dateExpected not in @data_esperada and dateStarted not in @data_iniciada'

# Retirando colunas que não salvaremos as informações
df_matches.drop(columns= ['homeTeamName', 'awayTeamName'], 
                inplace= True)

#df_matches= pd.concat([matches, df_matches])

df_matches= df_matches.query(query)
df_matches

,matchID,homeTeamID,awayTeamID,leagueID,round,season,week,day,dateExpected,dateStarted,homeTeamScore,awayTeamScore,supporters,weather,isFinished
17,43672367,51886,79085,51530,0,0,1172,5,2026-01-01 19:00,2026-01-01 19:00,8,1,3041,5,1
18,43669079,79085,6349,1001,3,80,1172,6,2026-01-02 21:59,2026-01-02 21:59,0,5,18200,3,1
19,43443566,79085,79672,2075,3,71,1173,1,2026-01-04 21:59,2026-01-04 21:59,3,0,11967,3,1
20,43655400,142266,79085,60882,2,1,1173,2,2026-01-05 19:15,2026-01-05 19:15,0,13,1156,0,1
21,43682280,141522,79085,51530,0,0,1173,3,2026-01-06 22:00,2026-01-06 22:00,1,3,2456,1,1
22,43443580,122510,79085,2075,4,71,1173,4,2026-01-07 21:59,2026-01-07 21:59,0,9,3011,5,1


In [5]:
if len(df_matches) > 0:
    
    # Adiciona dados à tabela
    df_matches.to_sql('matches', con=db, if_exists='append', index= False)

In [6]:
#df_matches = pd.read_sql_query("SELECT * FROM matches", db)
len(df_matches)

6

### Estatísticas dos jogos

In [7]:
#Pega informações únicas das estatísticas já adicionadas para que não sejam inseridos valores duplicados na base
stats = pd.read_sql_query("SELECT * FROM matchStats", db)

ids= stats['matchID'].unique()
team= stats['teamID'].unique()
terco_defensivo= stats['timeOnHalf'].unique()
posse_de_bola= stats['timePossession'].unique()

In [8]:
#lista que guardará as estatísticas de cada partida
list_stats= list()

# Pegando estatísticas

for ID in df_matches['matchID'].unique():
    
    url= f'https://sokker.org/xml/match-{ID}.xml'
    df= pega_dados(url, sessao, search= 'teamStats')
    df['matchID']= ID
    list_stats.append(df)
    


df_stats= pd.concat([stats for stats in list_stats])

# Retirando valores nulos (partidas ainda não jogadas) e duplicados (partidas já adicionadas na base)

df_stats.dropna(inplace= True)

query= 'matchID not in @ids and teamID not in @team and timeOnHalf not in @terco_defensivo and timePossession not in @posse_de_bola'

df_stats= df_stats.query(query)

df_stats

,teamID,timeOnHalf,timePossession,offsides,shoots,fouls,yellowCards,redCards,tacticName,ratingScoring,ratingPassing,ratingDefending,matchID
0,51886,24108,33587,4,22,1,0,0,Ronaldyno v2,12,11,10,43672367
1,79085,45852,36373,0,1,4,1,0,3-5-2ET,7,8,10,43672367
0,79085,55734,31377,0,0,0,0,0,4-3-3aRU,5,6,7,43669079
1,6349,14699,39056,5,23,1,0,0,4132 Ultra Press,9,10,7,43669079
0,79085,28302,36679,1,16,2,0,0,4-3-3aRU,9,7,9,43443566
1,79672,42364,33987,0,1,2,0,0,PPR1,7,5,6,43443566
0,142266,57888,26352,0,0,1,0,0,433,2,3,4,43655400
1,79085,12092,43628,0,39,0,0,0,4-4-2EF,9,7,9,43655400
0,141522,42209,34518,1,2,0,0,0,442,8,7,10,43682280
1,79085,27982,35673,0,11,1,1,0,4-4-2EF,9,7,10,43682280


In [9]:
# Adicionando informações se houver alguma nova
if len(df_stats) > 0:
    
    # Adiciona dados à tabela
    df_stats.to_sql('matchStats', con=db, if_exists='append', index= False)

In [10]:
# Conferindo
stats = pd.read_sql_query("SELECT * FROM matchStats", db)
len(stats)

2056

### Estatísticas da Liga

In [11]:
#Pega informações ligas já adicionadas para que não sejam inseridos valores duplicados na base
league_stats = pd.read_sql_query("SELECT * FROM leagueStats", db)

ids= league_stats['leagueID'].unique()
team= league_stats['teamID'].unique()
rodada=  league_stats['round'].unique()

In [12]:
#Pegar o id da liga desejada entrando na liga na página do sokker
league_id= 1349

url= f'https://sokker.org/xml/league-{league_id}.xml'
df_league_stats= pega_dados(url, sessao, search= 'teams', tree_top= True)

In [13]:
df_league_stats['leagueID']= league_id
df_league_stats

,teamID,round,points,wins,draws,losses,goalsScored,goalsLost,rankTotal,leagueID
0,90105,4,9,3,0,1,24,5,9519024003009,1349
1,23755,4,9,3,0,1,23,7,9516023003004,1349
2,79587,4,8,2,2,0,13,5,8508013002007,1349
3,78671,4,8,2,2,0,10,2,8508010002006,1349
4,78606,4,7,2,1,1,20,7,7513020002010,1349
5,89569,4,7,2,1,1,14,4,7510014002005,1349
6,124139,4,7,2,1,1,18,9,7509018002011,1349
7,78751,4,4,1,1,2,11,14,4497011001008,1349
8,79656,4,4,1,1,2,2,14,4488002001001,1349
9,121224,4,4,1,1,2,5,19,4486005001002,1349


In [14]:
# Adicionando informações 
if len(df_league_stats) > 0:
    
    # Adiciona dados à tabela
    df_league_stats.to_sql('leagueStats', con=db, if_exists='append', index= False)

league_stats = pd.read_sql_query("SELECT * FROM leagueStats", db)
len(league_stats)

### Liga informação geral

In [15]:
#Pega informações das ligas já adicionadas para que não sejam inseridos valores duplicados na base
leagues = pd.read_sql_query("SELECT * FROM league", db)

ids= leagues['leagueID'].unique()
league_name= leagues['name'].unique()

In [16]:
#Pegando dados da liga
df_league= pega_dados(url, sessao, search= 'info')

#Retirando coluna que não será utilizada
df_league.drop(columns= ['userID', 'round'], inplace= True)

#Retirando valores duplicados (se as estatísticas já tiverem sido adicionadas na base)
query= 'leagueID not in @ids and name not in @league_name'
df_league= df_league.query(query)

df_league

,leagueID,name,countryID,division,season,type,isOfficial,isCup


In [17]:
# Adicionando informações se for uma liga ainda não adicionada
if len(df_league) > 0:
    
    # Adiciona dados à tabela
    df_league.to_sql('league', con=db, if_exists='append', index= False)

In [18]:
league = pd.read_sql_query("SELECT * FROM league", db)
len(league)

4

In [19]:
# Salvando alterações
db.commit()

# Fechando conexão com a base de dados
db.close()